# Tutorial 3: Clipper in Action with Pong

We have already explored some of the features of Clipper (insert recap here). Now let's take a look at Clipper in action with Pong! Released by Atari in 1972, Pong was the first commercially successful video game. You can read more about it <a href="https://en.wikipedia.org/wiki/Pong">here</a>

The goal of this tutorial is to use Clipper to deploy several AI policies to play against, and in doing so explore:
(list of Clipper features we want to highlight)

This tutorial will be broken up into 3 main parts:
#### 1. Starting Clipper and Deploying First Policy
#### 2. Training a Better Model
#### 3. Deploying an updated, trained model

## Part 1: Starting Clipper and Deploying First Policy

Install packaged env from requirements.txt (not sure if we should do this before opening jupyter notebook or not)

In [1]:
#Import dependencies
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as py_deployer
import random
import numpy as np
import pandas as pd
from datascience import *
from sklearn import linear_model
from IPython.display import Image
from IPython.core.display import HTML 


Our first step is to get Clipper started and deploy our first AI policy: one that guesses which direction to move the paddle randomly. First, start Docker (or make sure it is running)

In [2]:
#Starting Clipper, make sure you have Docker running before you run this cell
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.stop_all()
clipper_conn.start_clipper()

18-09-05:21:37:26 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers
18-09-05:21:37:26 INFO     [docker_container_manager.py:119] Starting managed Redis instance in Docker
18-09-05:21:37:29 INFO     [clipper_admin.py:126] Clipper is running


Running the cell below will register an application in Clipper called "pong" and create a Clipper endpoint for our random policy at http://localhost:1337/pong/predict

In [3]:
def random_predict(xs):
    action = random.randint(0, 2)
    return [str(action) for _ in xs]

py_deployer.create_endpoint(clipper_conn, name="pong", input_type="doubles", func=random_predict,
                            default_output="0", slo_micros=100000)

18-09-05:21:37:38 INFO     [clipper_admin.py:201] Application pong was successfully registered
18-09-05:21:37:38 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmpmh1yuy5y
18-09-05:21:37:38 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-09-05:21:37:38 INFO     [python.py:192] Python closure saved
18-09-05:21:37:38 INFO     [python.py:202] Using Python 3.5 base image
18-09-05:21:37:38 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmpmh1yuy5y
18-09-05:21:37:38 INFO     [clipper_admin.py:456] {'stream': 'Step 1/2 : FROM clipper/python35-closure-container:0.3.0'}
18-09-05:21:37:38 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-05:21:37:38 INFO     [clipper_admin.py:456] {'stream': ' ---> 1fc910c1ed28\n'}
18-09-05:21:37:38 INFO     [clipper_admin.py:456] {'stream': 'Step 2/2 : COPY /tmp/clipper/tmpmh1yuy5y /model/'}
18-09-05:21:37:38 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-05

Now that we have a model deployed, let's see how it works! Run the cell below to start the game

In [4]:
!python pong-server.py localhost:1337

18-09-05:17:56:52 INFO     [pong-server.py:109] Starting Pong Server on localhost:3000
18-09-05:17:56:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/index.html
127.0.0.1 - - [05/Sep/2018 17:56:58] "GET /pong/ HTTP/1.1" 200 -
18-09-05:17:56:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/game.js
18-09-05:17:56:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/pong.css
18-09-05:17:56:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/pong.js
127.0.0.1 - - [05/Sep/2018 17:56:58] "GET /pong/pong.css HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2018 17:56:58] "GET /pong/pong.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2018 17:56:58] "GET /pong/game.js HTTP/1.1" 200 -
18-09-05:17:56:59 INFO     [pong-server.py:51] Local path: /Users/h

18-09-05:18:04:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/pong.css
127.0.0.1 - - [05/Sep/2018 18:04:58] "GET /pong/pong.css HTTP/1.1" 200 -
18-09-05:18:04:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/game.js
127.0.0.1 - - [05/Sep/2018 18:04:58] "GET /pong/game.js HTTP/1.1" 200 -
18-09-05:18:04:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/pong.js
127.0.0.1 - - [05/Sep/2018 18:04:58] "GET /pong/pong.js HTTP/1.1" 200 -
18-09-05:18:04:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/images/press1.png
18-09-05:18:04:58 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/images/press2.png
18-09-05:18:04:58 INFO     [pong-server.py:51] Local path: /Us

18-09-05:18:08:28 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/images/press1.png
18-09-05:18:08:28 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/images/press2.png
18-09-05:18:08:28 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/images/winner.png
127.0.0.1 - - [05/Sep/2018 18:08:28] "GET /pong/images/press1.png HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2018 18:08:28] "GET /pong/images/press2.png HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2018 18:08:28] "GET /pong/images/winner.png HTTP/1.1" 200 -
18-09-05:18:08:28 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/sounds/ping.wav
18-09-05:18:08:28 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/sounds/wall.wav
18-09-05

18-09-05:18:10:45 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/sounds/pong.wav
18-09-05:18:10:45 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/sounds/ping.wav
127.0.0.1 - - [05/Sep/2018 18:10:45] "GET /pong/sounds/ping.wav HTTP/1.1" 200 -
18-09-05:18:10:45 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/sounds/wall.wav
127.0.0.1 - - [05/Sep/2018 18:10:45] "GET /pong/sounds/wall.wav HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2018 18:10:45] "GET /pong/sounds/pong.wav HTTP/1.1" 200 -
18-09-05:18:10:45 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/sounds/goal.wav
127.0.0.1 - - [05/Sep/2018 18:10:45] "GET /pong/sounds/goal.wav HTTP/1.1" 200 -
18-09-05:18:10:45 INFO     [pong-server.py:51] Local path: /favicon.ico
127.0.0.1 - - [0

Congratulations! We have depolyed our first model to Clipper! Let's take a look at it by clicking http://localhost:3000/pong/ and pressing 1 to start the game.

## Part 2: Training a better model

As you probably noticed, the random-guessing policy did not perform well at all. In order to train a better model, we are going to use imitation learning. Let's play the game again, this time with 2 actual players, and we will collect data on how you play the game to train our model! 

Go to http://localhost:3000/pong/ and press 2. Find someone next to you and play pong against them!!

<b>Part to collect data into a output.csv --> currently doing this in really hacky way to just get it to work, but trying to figure out better solution</b>

Now that we have our data, lets train a new model! First, run the cell below to clean the data and format it for scikit-learn's LogisticRegression model.

In [4]:
#Read data into table
data_table= Table.read_table('out.csv')

#Convert labels into numeric values
def convert_label(label):
    if(label=="down"):
        return 1
    elif(label=="up"):
        return 2
    else:
        return 0
data_table_2 = data_table.with_column("label", data_table.apply(convert_label, 0))

#Scale our data down
final_table = Table().with_columns("label", data_table.apply(convert_label, 0),
                                  "paddle_y", data_table.column(1)/500.0,
                                  "ball_x", data_table.column(2)/500.0,
                                  "ball_y", data_table.column(3)/500.0,
                                  "ball_dx", data_table.column(4)/500.0,
                                  "ball_dy", data_table.column(5)/500.0,
                                  "x_prev", data_table.column(6)/500.0,
                                  "y_prev", data_table.column(7)/500.0)
data_df = final_table.to_df()
data_df.take(np.arange(5))

,label,paddle_y,ball_x,ball_y,ball_dx,ball_dy,x_prev,y_prev
0,1,0.420000,0.398852,0.622737,0.334168,0.334168,0.382163,0.606048
1,1,0.638988,0.586093,0.809978,0.343016,0.343016,0.569304,0.793190
2,1,0.678588,0.620475,0.844360,0.344616,0.344616,0.602576,0.826461
3,1,0.718188,0.655016,0.878901,0.346216,0.346216,0.637725,0.861610
4,1,0.738384,0.672694,0.896579,0.347032,0.347032,0.655016,0.878901


We are going to use the data to train a scikit-learn Logistic Regression model. You can read more about the particular model <a url="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">here</a>

In [5]:
labels = data_df['label']
training_data= data_df.drop(['label'], axis=1)

model = linear_model.LogisticRegression()
model.fit(training_data, labels)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Part 3: Deploying updated model

Now that we have an updated model, we can deploy that model onto Clipper. Once the system realizes there is a new version of the model, it will automatically switch to serving the newer version of the model.

In [6]:
Image(url= "update_model.png")

In [7]:
def predict(inputs):
    # model.predict returns a list of predictions
    preds = model.predict(inputs)
    return [str(p) for p in preds]

py_deployer.deploy_python_closure(clipper_conn, name="pong", version=3, input_type="doubles", func=predict, pkgs_to_install=["numpy","scipy", "pandas", "datascience", "sklearn"])


18-09-05:21:37:51 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmpso5u2nzd
18-09-05:21:37:51 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-09-05:21:37:51 INFO     [python.py:192] Python closure saved
18-09-05:21:37:51 INFO     [python.py:202] Using Python 3.5 base image
18-09-05:21:37:51 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmpso5u2nzd
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': 'Step 1/3 : FROM clipper/python35-closure-container:0.3.0'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': ' ---> 1fc910c1ed28\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': 'Step 2/3 : COPY /tmp/clipper/tmpso5u2nzd /model/'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': ' ---> 9ddc5afe85bf\n'}
18-09-05:21:38:39 

18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': 'Collecting babel!=2.0,>=1.3 (from sphinx->datascience)\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/b8/ad/c6f60602d3ee3d92fbed87675b6fb6a6f9a38c223343ababdb44ba201f10/Babel-2.6.0-py2.py3-none-any.whl (8.1MB)\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': 'Collecting alabaster<0.8,>=0.7 (from sphinx->datascience)\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/6e/71/c3648cc2f675063dbe2d669004a59e4a5120172713a1de3c3b14144d4b31/alabaster-0.7.11-py2.py3-none-any.whl\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': 'Collecting Pygments>=2.0 (from sphinx->datascience)\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/02/ee/b6e02dc6529e82b75bb06823ff7d005b141037cb1416b10c6f00fc419dca/P

18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': 'Successfully installed Pygments-2.2.0 alabaster-0.7.11 atomicwrites-1.2.1 attrs-18.2.0 babel-2.6.0 coverage-3.7.1 coveralls-0.5 datascience-0.10.5 docopt-0.6.2 docutils-0.14 folium-0.2.1 imagesize-1.1.0 more-itertools-4.3.0 packaging-17.1 pandas-0.23.4 pathlib2-2.3.2 pluggy-0.7.1 py-1.6.0 pyparsing-2.2.0 pytest-3.7.4 pytz-2018.5 scikit-learn-0.19.2 scipy-1.1.0 sklearn-0.0 snowballstemmer-1.2.1 sphinx-1.7.9 sphinxcontrib-websupport-1.1.0\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': "\x1bYou are using pip version 9.0.3, however version 18.0 is available.\nYou should consider upgrading via the 'pip install --upgrade pip' command.\n\x1b"}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'stream': ' ---> 5337c603abfc\n'}
18-09-05:21:38:39 INFO     [clipper_admin.py:456] {'aux': {'ID': 'sha256:5337c603abfcc9314736d484ad45ae5f919a8758c5189d4bacc201b211b538aa'}}
18-09-05:21:38:39 INFO     [clipper_admin.py:456]

You may notice that it seems like we only deployed the predict function to Clipper. However, Clipper will track the dependencies that the model in the predict function uses and include them in the model Docker container as dependencies. You can take a look at the logs above to see the Docker container downloading said dependencies.

Let's run our server again (or refresh the page if you haven't stopped the server previously)

In [8]:
!python pong-server.py localhost:1337

18-09-05:21:38:56 INFO     [pong-server.py:109] Starting Pong Server on localhost:3000
18-09-05:21:39:04 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/index.html
127.0.0.1 - - [05/Sep/2018 21:39:04] "GET /pong/ HTTP/1.1" 200 -
18-09-05:21:39:04 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/pong.css
127.0.0.1 - - [05/Sep/2018 21:39:04] "GET /pong/pong.css HTTP/1.1" 200 -
18-09-05:21:39:04 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/game.js
18-09-05:21:39:04 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/risecamp/rl_and_pong/javascript-pong/static/pong.js
127.0.0.1 - - [05/Sep/2018 21:39:04] "GET /pong/pong.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2018 21:39:04] "GET /pong/game.js HTTP/1.1" 200 -
18-09-05:21:39:05 INFO     [pong-server.py:51] Local path: /Users/h

<_io.BufferedReader name=10>
18-09-05:21:39:09 INFO     [pong-server.py:73] 114
114
18-09-05:21:39:09 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.5784, 0.185022432, 0.7212747620394769, 0.32376799999999983, 0.32376799999999983, 0.1695, 0.705752330039477]}
127.0.0.1 - - [05/Sep/2018 21:39:09] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":10,"output":1,"default":false}' in 93.173 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:39:09 INFO     [pong-server.py:73] 138
138
18-09-05:21:39:09 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.6001799999999999, 0.20285387199999996, 0.7391062020394769, 0.3246479999999999, 0.3246479999999999, 0.185022432, 0.7212747620394769]}
<_io.BufferedReader name=12>
18-09-05:21:39:09 INFO     [pong-server.py:73] 136
136
18-09-05:21:39:09 INFO     [pong-server.py:75] http:

<_io.BufferedReader name=8>
18-09-05:21:39:10 INFO     [pong-server.py:73] 114
114
18-09-05:21:39:10 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.816, 0.41591500799999975, 0.91097432, 0.33498399999999984, -0.33498399999999984, 0.4008569279999998, 0.926]}
127.0.0.1 - - [05/Sep/2018 21:39:10] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":25,"output":2,"default":false}' in 21.624000000000002 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:39:10 INFO     [pong-server.py:73] 124
124
18-09-05:21:39:10 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.795804, 0.43301999999999974, 0.893910944, 0.33579999999999977, -0.33579999999999977, 0.41591500799999975, 0.91097432]}
127.0.0.1 - - [05/Sep/2018 21:39:10] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":26,"output":2,"default":false}

127.0.0.1 - - [05/Sep/2018 21:39:11] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":39,"output":2,"default":false}' in 20.428 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:39:11 INFO     [pong-server.py:73] 137
137
18-09-05:21:39:11 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.519396, 0.6713060319999995, 0.6561821920000003, 0.34696799999999967, -0.34696799999999967, 0.6546699999999995, 0.6727813600000004]}
127.0.0.1 - - [05/Sep/2018 21:39:11] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":40,"output":2,"default":false}' in 19.035 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:39:11 INFO     [pong-server.py:73] 148
148
18-09-05:21:39:11 INFO     [pong-server.py:75] http://localhost:1337/pong/pre

18-09-05:21:39:11 INFO     [pong-server.py:73] 135
135
18-09-05:21:39:11 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.241404, 0.9188199999999992, 0.4092321920000005, 0.3581999999999996, -0.3581999999999996, 0.9009299999999991, 0.4270821920000005]}
127.0.0.1 - - [05/Sep/2018 21:39:11] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":54,"output":2,"default":false}' in 19.525000000000002 ms
{'Content-Type': 'application/json', 'Content-Length': '42'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:39:11 INFO     [pong-server.py:73] 135
135
18-09-05:21:39:11 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.220416, 0.9378270719999992, 0.3902700640000005, 0.3590479999999996, -0.3590479999999996, 0.9188199999999992, 0.4092321920000005]}
127.0.0.1 - - [05/Sep/2018 21:39:11] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":55,"output":2,"default":fa

...

<_io.BufferedReader name=10>
18-09-05:21:40:20 INFO     [pong-server.py:73] 121
121
18-09-05:21:40:20 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.816, 1.0383123999999988, 0.7102653999999993, 0.4142479999999996, 0.227524, 1.0163797279999987, 0.6982290999999994]}
127.0.0.1 - - [05/Sep/2018 21:40:20] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1428,"output":1,"default":false}' in 83.689 ms
{'Content-Type': 'application/json', 'Content-Length': '44'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:40:20 INFO     [pong-server.py:73] 133
133
18-09-05:21:40:20 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.816, 1.0594598559999988, 0.7218899319999994, 0.41506399999999954, 0.22834000000000002, 1.0383123999999988, 0.7102653999999993]}
127.0.0.1 - - [05/Sep/2018 21:40:20] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1429,"output":1,"defaul

127.0.0.1 - - [05/Sep/2018 21:40:21] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1443,"output":1,"default":false}' in 50.969 ms
{'Content-Type': 'application/json', 'Content-Length': '44'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:40:21 INFO     [pong-server.py:73] 114
114
18-09-05:21:40:21 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.816, 1.140570848, 0.8553711459999993, -0.4261999999999995, 0.12173, 1.1605492320000002, 0.8496675079999993]}
127.0.0.1 - - [05/Sep/2018 21:40:21] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1444,"output":1,"default":false}' in 21.494 ms
{'Content-Type': 'application/json', 'Content-Length': '44'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:40:21 INFO     [pong-server.py:73] 126
126
18-09-05:21:40:21 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.8

18-09-05:21:40:22 INFO     [pong-server.py:73] 129
129
18-09-05:21:40:22 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.7756080000000001, 0.8161488960000007, 0.9058465659999999, -0.4382319999999993, -0.133762, 0.8388720640000007, 0.912737294]}
127.0.0.1 - - [05/Sep/2018 21:40:22] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1459,"output":2,"default":false}' in 30.831 ms
{'Content-Type': 'application/json', 'Content-Length': '44'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:40:22 INFO     [pong-server.py:73] 129
129
18-09-05:21:40:22 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.7562040000000001, 0.7946947360000008, 0.899311436, -0.4390159999999993, -0.134546, 0.8161488960000007, 0.9058465659999999]}
127.0.0.1 - - [05/Sep/2018 21:40:22] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1460,"output":2,"default":false}' in 23.074 ms
{

<_io.BufferedReader name=8>
18-09-05:21:40:22 INFO     [pong-server.py:73] 148
148
18-09-05:21:40:22 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.4797960000000001, 0.48492128000000106, 0.8020580399999998, -0.45018399999999925, -0.14571400000000007, 0.506026912000001, 0.8088535819999998]}
127.0.0.1 - - [05/Sep/2018 21:40:22] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1474,"output":2,"default":false}' in 14.48 ms
{'Content-Type': 'application/json', 'Content-Length': '44'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:40:22 INFO     [pong-server.py:73] 149
149
18-09-05:21:40:22 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.4588080000000001, 0.4610840000000011, 0.7943576699999998, -0.45103199999999927, -0.14656200000000008, 0.48492128000000106, 0.8020580399999998]}
127.0.0.1 - - [05/Sep/2018 21:40:22] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded 

127.0.0.1 - - [05/Sep/2018 21:40:23] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1488,"output":2,"default":false}' in 23.236 ms
{'Content-Type': 'application/json', 'Content-Length': '44'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:40:23 INFO     [pong-server.py:73] 136
136
18-09-05:21:40:23 INFO     [pong-server.py:75] http://localhost:1337/pong/predict
{'input': [0.1824, 0.14292401600000154, 0.6887177459999994, -0.4621999999999992, -0.15773000000000015, 0.1650543200000015, 0.6962334899999993]}
127.0.0.1 - - [05/Sep/2018 21:40:23] "POST /pong/predict HTTP/1.1" 200 -
Clipper responded with '{"query_id":1489,"output":2,"default":false}' in 49.549 ms
{'Content-Type': 'application/json', 'Content-Length': '44'}
<class 'requests.structures.CaseInsensitiveDict'>
<_io.BufferedReader name=8>
18-09-05:21:40:23 INFO     [pong-server.py:73] 140
140
18-09-05:21:40:23 INFO     [pong-server.py:75] http://localhost:1337/pong/

Cool points here:
1. We never restarted the pong server, but our code seamlessly switches to serving the updated model

## 4. Conclusion

Just as a recap, here's what we did today:

1. Deployed an initial random policy to Clipper and served predictions
2. Trained a new model with data that you generated while playing pong
3. Deployed new version of model and seamlessly switch the model being served

By doing so, we've explored the following Clipper features:

1. Deploy models trained in your choice of framework to Clipper with a few lines of code by using an existing model container or writing your own
2. Easily update or roll back models in running applications
3. Run each model in a separate Docker container for simple cluster management and resource allocation


In [10]:
!python stop_c.py


18-09-05:21:54:05 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers


# TO-DOS

1. Figure out a more seamless transition between training data and stuff
    - Node server
    - Somehow saving the console
    - other ideas appreciated
2. Fix copy on tutorial
3. More content(introduce rollbacks for older models?)
4. Time concerns